# mecano_extra_se - atac_seq
This notebook will create all the necessary files, scripts and folders to pre-process the aforementioned project. Is designed to be used in a jupyter server deployed in a system running SLURM. The majority of the scripts and heavy-lifting processes are wrapped up in sbatch scripts.As an end user, in order to pre-process your samples provided in the spread sheet, you will simply need to *run the entire notebook* (Cell > Run all) and the system should take care of the rest for you.
#### Create necessary folder(s)

In [ ]:
%%bash
mkdir -p /data/reddylab/Alex/collab/20240318_Crawford/data/atac_seq/metadata
mkdir -p /data/reddylab/Alex/collab/20240318_Crawford/data/atac_seq/raw_reads
mkdir -p /data/reddylab/Alex/collab/20240318_Crawford/data/atac_seq/processed_raw_reads
mkdir -p /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts
mkdir -p /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/jsons
mkdir -p /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/logs

Save metadata file

In [1]:
%%writefile /data/reddylab/Alex/collab/20240318_Crawford/data/atac_seq/metadata/atac_seq_download_metadata.mecano_extra_se.txt
Sequencing core project	Sequencing core library name	Name	Paired-end or single-end	Genome	Library type	Umis	Control	Strand specificity	Blacklist removal	With ercc spike-in
mecano_extra	G2911	lung_fibroblast.atacseq.IPF.brep1	SE	hg38	ATAC-seq				True	
mecano_extra	G2912	lung_fibroblast.atacseq.IPF.brep2	SE	hg38	ATAC-seq				True	
mecano_extra	G2913	lung_fibroblast.atacseq.IPF.brep3	SE	hg38	ATAC-seq				True	
mecano_extra	G2914	lung_fibroblast.atacseq.IPF.brep4	SE	hg38	ATAC-seq				True	
mecano_extra	G2919	A549.atacseq.gel_1kpa.brep1	SE	hg38	ATAC-seq				True	
mecano_extra	G2920	A549.atacseq.gel_1kpa.brep2	SE	hg38	ATAC-seq				True	
mecano_extra	G2921	A549.atacseq.gel_1kpa.brep3	SE	hg38	ATAC-seq				True	
mecano_extra	G2928	HCT116.atacseq.gel_1kpa.brep1	SE	hg38	ATAC-seq				True	
mecano_extra	G2929	HCT116.atacseq.gel_1kpa.brep2	SE	hg38	ATAC-seq				True	
mecano_extra	G2930	HCT116.atacseq.gel_1kpa.brep3	SE	hg38	ATAC-seq				True	
mecano_extra	G2922	A549.atacseq.gel_50kpa.brep1	SE	hg38	ATAC-seq				True	
mecano_extra	G2923	A549.atacseq.gel_50kpa.brep2	SE	hg38	ATAC-seq				True	
mecano_extra	G2924	A549.atacseq.gel_50kpa.brep3	SE	hg38	ATAC-seq				True	
mecano_extra	G2925	HCT116.atacseq.gel_50kpa.brep1	SE	hg38	ATAC-seq				True	
mecano_extra	G2926	HCT116.atacseq.gel_50kpa.brep2	SE	hg38	ATAC-seq				True	
mecano_extra	G2927	HCT116.atacseq.gel_50kpa.brep3	SE	hg38	ATAC-seq				True	
mecano_extra	G2915	lung_fibroblast.atacseq.control.brep1	SE	hg38	ATAC-seq				True	
mecano_extra	G2916	lung_fibroblast.atacseq.control.brep2	SE	hg38	ATAC-seq				True	
mecano_extra	G2917	lung_fibroblast.atacseq.control.brep3	SE	hg38	ATAC-seq				True	
mecano_extra	G2918	lung_fibroblast.atacseq.control.brep4	SE	hg38	ATAC-seq				True	


Writing /data/reddylab/Alex/collab/20240318_Crawford/data/atac_seq/metadata/atac_seq_download_metadata.mecano_extra_se.txt


### FASTQ files already available locally!!
Please, make sure the FASTQ files are correctly named, decompressed and located/symlinked in:

**
/data/reddylab/Alex/collab/20240318_Crawford/data/atac_seq/processed_raw_reads/mecano_extra_se
**

In [6]:
%%bash
# mkdir -p /data/reddylab/Alex/collab/20240318_Crawford/data/atac_seq/processed_raw_reads/mecano_extra_se
cd /data/reddylab/Alex/collab/20240318_Crawford/data/atac_seq/processed_raw_reads/mecano_extra_se
# ln -s ../mecano_extra/*.R1.fastq.gz ./
rename .R1.fastq .fastq *.R1.fastq.gz

#### Create JSON files for CWL pipeline files

In [3]:
%%writefile /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/cwl_json_gen_mecano_extra_se.sh
#!/bin/bash
ORDER=mecano_extra_se
PROCESSED_DATA_DIR=/data/reddylab/Alex/collab/20240318_Crawford/data/atac_seq/processed_raw_reads/${ORDER}
METADATA=/data/reddylab/Alex/collab/20240318_Crawford/data/atac_seq/metadata/atac_seq_download_metadata.mecano_extra_se.txt

python /data/reddylab/software/cwl/GGR-cwl/v1.0/json-generator/run.py \
    -m ${METADATA} \
    -d ${PROCESSED_DATA_DIR} \
    -o /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/jsons \
    -t atac-seq \
    --fastq-gzipped \
    --mem 24000 \
    --nthreads 16 \
    --separate-jsons



Writing /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/cwl_json_gen_mecano_extra_se.sh


Execute file to create JSON files

In [4]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -o /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/logs/mecano_extra_se_cwl_json_gen.out \
 -p all \
 /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/cwl_json_gen_mecano_extra_se.sh

Extract blocking job id

In [5]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create SLURM array master bash file for se-blacklist-removal samples

In [7]:
%%writefile /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/mecano_extra_se-se-blacklist-removal.sh
#!/bin/bash
#SBATCH --job-name=cwl_atac_seq
#SBATCH --output=/data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/logs/mecano_extra_se-se-blacklist-removal-%a.out
#SBATCH --mail-user=aeb84@duke.edu
#SBATCH --mail-type=FAIL,END
#SBATCH --mem=24000
#SBATCH --cpus-per-task=16

export PATH="/data/reddylab/software/bin:$PATH"
export PATH="/data/common/shared_conda_envs/ucsc/bin:$PATH"
export PATH="/data/reddylab/software/cwl/bin:$PATH"
export PATH="/data/reddylab/software/preseq_v2.0:$PATH"
export PATH="/data/reddylab/software/rsem-1.2.21/:$PATH"
export PATH="/data/reddylab/software/phantompeakqualtools-1.2/:$PATH"
export PATH="/data/reddylab/software/miniconda2/envs/cwl10/bin:$PATH"

module load bedtools2
module load fastqc
module load samtools
module load bowtie2
module load java

# For Fastqc
export DISPLAY=:0.0

# Make sure temporary files and folders are created in a specific folder
mkdir -p /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/tmpdirs/tmp-mecano_extra_se-se-blacklist-removal-${SLURM_ARRAY_TASK_ID}-
export TMPDIR="/data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/tmpdirs/tmp-mecano_extra_se-se-blacklist-removal-${SLURM_ARRAY_TASK_ID}-"

cwltool --debug \
    --non-strict \
    --preserve-environment PATH \
    --preserve-environment DISPLAY \
    --preserve-environment TMPDIR \
    --outdir /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/mecano_extra_se-se-blacklist-removal  \
    --no-container \
    /data/reddylab/software/cwl/GGR-cwl/v1.0/ATAC-seq_pipeline/pipeline-se-blacklist-removal.cwl \
    /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/jsons/atac_seq_download_metadata.mecano_extra_se-se-blacklist-removal-${SLURM_ARRAY_TASK_ID}.json

# Delete any tmpdir not removed by cwltool
rm -rf /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/tmpdirs/tmp-mecano_extra_se-se-blacklist-removal-${SLURM_ARRAY_TASK_ID}-


Writing /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/mecano_extra_se-se-blacklist-removal.sh


In [ ]:
macs2 \
    callpeak \
    -n \
    /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/mecano_extra_se-se-blacklist-removal/A549.atacseq.gel_1kpa.brep3.trimmed.masked.dedup.sorted \
    --bdg \
    --extsize \
    200 \
    -f \
    BAM \
    -g \
    hs \
    --nomodel \
    -q \
    0.1 \
    --shift \
    -100 \
    --treatment \
    /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/mecano_extra_se-se-blacklist-removal/A549.atacseq.gel_1kpa.brep3.trimmed.masked.dups_marked.bam

Execute SLURM array master file

In [8]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -p all \
 --depend afterok:$1 \
 --array 0-19%20 \
 /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/mecano_extra_se-se-blacklist-removal.sh

Extract blocking job id

In [9]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create QC generating script

In [2]:
%%writefile /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/generate_qc_cell_mecano_extra_se-se-blacklist-removal.sh
#!/bin/bash
#SBATCH --job-name=qc
#SBATCH --output=/data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/logs/qc_gen.mecano_extra_se-se-blacklist-removal.out

source /data/reddylab/software/miniconda2/bin/activate alex
cd /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/mecano_extra_se-se-blacklist-removal

python /data/reddylab/software/cwl/bin/generate_stats_atacseq_single_end.py ./ \
    -samples $(/bin/ls -1 *PBC.txt | sed 's@.trimmed.PBC.txt@@') \
> qc.txt

Overwriting /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/generate_qc_cell_mecano_extra_se-se-blacklist-removal.sh


Generate QCs for mecano_extra_se-se-blacklist-removal

In [11]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -p all \
 --depend afterok:$1 \
 /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/generate_qc_cell_mecano_extra_se-se-blacklist-removal.sh

Add ATAQC

In [9]:
%%bash
module load bedtools2
source /data/reddylab/software/miniconda2/bin/activate alex_dev
BAMFILES=($(/bin/ls -1 /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/mecano_extra_se-se-blacklist-removal/*bam))
#     --array=1-$((${#BAMFILES[@]}-1)) \
sbatch -p all \
    --cpus-per-task 8 \
    --array=1-$((${#BAMFILES[@]}-1)) \
    -o /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/logs/ataqc.mecano_extra_se-se-blacklist-removal.%a.out \
    <<'EOF'
#!/bin/bash
BAMFILES=($(/bin/ls -1 /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/mecano_extra_se-se-blacklist-removal/*bam))
BAM=${BAMFILES[$SLURM_ARRAY_TASK_ID]}
# 
python /data/reddylab/Alex/reddylab_utils/scripts/ataqc.make_tss_plot.py \
    --bam ${BAM} \
    --tss /data/reddylab/Reference_Data/Refseq/GRCh38/GRCh38_latest_genomic.ucsc_chromosomes.promoters.protein_coding.no_alt.bed \
    --read-length 100 \
    --processes 8 \
    --outfile-prefix ${BAM/.bam/.ataqc} \
    --chromosome-sizes \
        /data/reddylab/Reference_Data/Genomes/hg38/GCA_000001405.15_GRCh38_no_alt_analysis_set.sizes
EOF

Submitted batch job 30719204


In [11]:
%%bash
ls -a /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/mecano_extra_se-se-blacklist-removal/*.ataqc_large_tss-enrich.png

/data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/mecano_extra_se-se-blacklist-removal/A549.atacseq.gel_1kpa.brep1.trimmed.masked.dups_marked.ataqc_large_tss-enrich.png


Extract blocking job id

In [ ]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

Add the re-process of one set of samples from Brian's original processing effort

In [5]:
%%writefile /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/jsons/atac_seq_download_metadata.COSGROVE_5109_180815B1-se-blacklist-removal-1.json
{
    "input_fastq_files": [
        { "class": "File", "path": "/data/gersbachlab/gersbach_portunus/cosgrove_brian/BDC04_ATACseq_RerunNewPipeline/data/atac_seq/processed_raw_reads/COSGROVE_5109_180815B1/compressed/HNF.BDC04atacseq.12kPa.20h.rep2.fastq.gz" }
    ],
    "as_narrowPeak_file": { "class": "File", "path": "/data/reddylab/Reference_Data/ENCODE/kent/src/hg/lib/encode/narrowPeak.as" },
    "genome_effective_size": "hs",
    "default_adapters_file": { "class": "File", "path": "/data/reddylab/Reference_Data/Adapters/default_adapters.fasta" },
    "genome_ref_first_index_file": { "class": "File", "path": "/data/reddylab/Reference_Data/Genomes/hg38/GCA_000001405.15_GRCh38_no_alt_analysis_set.1.ebwt"},
    "genome_sizes_file": { "class": "File", "path": "/data/reddylab/Reference_Data/Genomes/hg38/GCA_000001405.15_GRCh38_no_alt_analysis_set.sizes" },
    "ENCODE_blacklist_bedfile": { "class": "File", "path": "/data/reddylab/Reference_Data/ENCODE/hg38.blacklist.merged.bed" },
    "nthreads_qc": 16,
    "nthreads_trimm": 16,
    "nthreads_map": 16,
    "nthreads_peakcall": 16,
    "nthreads_quant": 16,
    "trimmomatic_jar_path": "/data/reddylab/software/Trimmomatic-0.32/trimmomatic-0.32.jar",
    "trimmomatic_java_opts": "-Xms12000m -Xmx24000m",
    "picard_jar_path": "/data/reddylab/software/picard-2.14.0/bin/picard.jar",
    "picard_java_opts": "-Xms12000m -Xmx24000m"
}

Writing /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/jsons/atac_seq_download_metadata.COSGROVE_5109_180815B1-se-blacklist-removal-1.json


In [13]:
%%writefile /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/COSGROVE_5109_180815B1-se-blacklist-removal.sh
#!/bin/bash
#SBATCH --job-name=cwl_atac_seq
#SBATCH --output=/data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/logs/COSGROVE_5109_180815B1-se-blacklist-removal-%a.out
#SBATCH --mail-user=alejandro.barrera@duke.edu
#SBATCH --mail-type=FAIL,END
#SBATCH --mem=24000
#SBATCH --cpus-per-task=16

export PATH="/data/reddylab/software/bin:$PATH"
export PATH="/data/common/shared_conda_envs/ucsc/bin:$PATH"
export PATH="/data/reddylab/software/cwl/bin:$PATH"
export PATH="/data/reddylab/software/preseq_v2.0:$PATH"
export PATH="/data/reddylab/software/rsem-1.2.21/:$PATH"
export PATH="/data/reddylab/software/phantompeakqualtools-1.2/:$PATH"
export PATH="/data/reddylab/software/miniconda2/envs/cwl10/bin:$PATH"

module load bedtools2
module load fastqc
module load samtools
module load bowtie2
module load java

# For Fastqc
export DISPLAY=:0.0

# Make sure temporary files and folders are created in a specific folder
mkdir -p /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/tmpdirs/tmp-COSGROVE_5109_180815B1-se-blacklist-removal-${SLURM_ARRAY_TASK_ID}-
export TMPDIR="/data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/tmpdirs/tmp-COSGROVE_5109_180815B1-se-blacklist-removal-${SLURM_ARRAY_TASK_ID}-"

cwltool --debug \
    --non-strict \
    --preserve-environment PATH \
    --preserve-environment DISPLAY \
    --preserve-environment TMPDIR \
    --outdir /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/COSGROVE_5109_180815B1-se-blacklist-removal  \
    --no-container \
    /data/reddylab/software/cwl/GGR-cwl/v1.0/ATAC-seq_pipeline/pipeline-se-blacklist-removal.cwl \
    /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/jsons/atac_seq_download_metadata.COSGROVE_5109_180815B1-se-blacklist-removal-${SLURM_ARRAY_TASK_ID}.json

Overwriting /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/COSGROVE_5109_180815B1-se-blacklist-removal.sh


In [15]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -p all \
 --array 10 \
 /data/reddylab/Alex/collab/20240318_Crawford/processing/atac_seq/scripts/COSGROVE_5109_180815B1-se-blacklist-removal.sh